# A Reproducibility study of Apnea Detection

Group 2:
- Ernest Onifade
- Harry Setiawan Hamjaya
- Mariama Oliveira

This notebook is a reproducibility study of the paper "Toward sleep apnea detection with lightweight multi-scaled fusion network." by Chen et al. The main goal of the paper is to develop a lightweight model able detect apnea based on single lead ECG recordings.
<br>
<br>
Link to the code of the original study : https://github.com/Bettycxh/Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network/tree/main

## Installing libraries

In [2]:
! pip install biosppy
! pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.6 MB/s eta 0:00:00


In [3]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

## Downloading dataset

In [ ]:
!wget https://physionet.org/static/published-projects/apnea-ecg/apnea-ecg-database-1.0.0.zip

--2023-10-24 20:10:45--  https://physionet.org/static/published-projects/apnea-ecg/apnea-ecg-database-1.0.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 608841016 (581M) [application/zip]
Saving to: ‘apnea-ecg-database-1.0.0.zip’

                      1%[                    ]   7.19M   941KB/s    eta 10m 28s

In [ ]:
!unzip apnea-ecg-database-1.0.0.zip

## Preprocessing

In this step, two main tasks were performed:

- Applying filters to the ECG signal in order to remove the noise
- Feature extraction: R peak amplitudes and R‐R intervals


Defining functions and applying tranformation to the training dataset

In [ ]:
# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
base_dir = "apnea-ecg-database-1.0.0"

fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300
num_worker = 35
# num_worker = 35 if cpu_count() > 35 else cpu_count() - 1  # Setting according to the number of CPU cores


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    # print('Preprocessing for train dataset...')
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



In [ ]:
# https://drive.google.com/file/d/199Hj2CpUMg-W6__fE7wB-0edcjMnVtSu/view?usp=sharing

# from google.colab import drive

# answer_id = '199Hj2CpUMg-W6__fE7wB-0edcjMnVtSu'

# answer_txt = f'https://drive.google.com/uc?id={answer_id}'

Applying transformation to the test dataset

In [ ]:
answers = {}
# with open(os.path.join("event-2-answers.txt"), "r") as f:
with open(os.path.join("event-2-answers.txt"), "r") as f:
    for answer in f.read().split("\n\n"):
        answers[answer[:3]] = list("".join(answer.split()[2::2]))

names = [
    "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
    "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
    "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
    "x31", "x32", "x33", "x34", "x35"
]

o_test = []
y_test = []
groups_test = []
print("Testing...")
with ProcessPoolExecutor(max_workers=num_worker) as executor:
    task_list = []
    for i in range(len(names)):
        labels = answers[names[i]]
        task_list.append(executor.submit(worker, names[i], labels))

    for task in as_completed(task_list):
        X, y, groups = task.result()
        o_test.extend(X)
        y_test.extend(y)
        groups_test.extend(groups)



Testing...
x12: 100%|██████████| 527/527 [1:02:40<00:00,  7.14s/it]


Saving features into a pickle file

In [ ]:
apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                  groups_test=groups_test)
with open(os.path.join("apnea-ecg.pkl"), "wb") as f:
    pickle.dump(apnea_ecg, f, protocol=2)

print("\nok!")


ok!


## Implementing the proposed model (E‐MSCNN)

In [ ]:
"""NOTES: Batch data is different each time in keras, which result in slight differences in results."""
import pickle
import keras
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from keras.layers import Dropout, MaxPooling1D, Reshape, multiply, Conv1D, GlobalAveragePooling1D, Dense, Input
from keras.models import Model, load_model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from scipy.interpolate import splev, splrep
from sklearn.metrics import confusion_matrix, f1_score
import random

base_dir = "apnea-ecg-database-1.0.0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
ir = 3  # interpolate interval
before = 2
after = 2
# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

def load_data(path):
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))
    # with open(os.path.join(base_dir, path), 'rb') as f:
    with open(os.path.join(path), 'rb') as f:  # read preprocessing result
        apnea_ecg = pickle.load(f)
    x_train1,x_train2,x_train3 = [],[],[]
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]
    groups_train = apnea_ecg["groups_train"]
    for i in range(len(o_train)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_train[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_train1.append([rri_interp_signal, ampl_interp_signal])  # 5-minute-long segment
        x_train2.append([rri_interp_signal[180:720], ampl_interp_signal[180:720]])  # 3-minute-long segment
        x_train3.append([rri_interp_signal[360:540], ampl_interp_signal[360:540]])  # 1-minute-long segment
    x_training1,x_training2,x_training3,y_training,groups_training = [],[],[],[],[]
    x_val1,x_val2,x_val3,y_val,groups_val = [],[],[],[],[]

    trainlist = random.sample(range(len(o_train)),int(len(o_train)*0.7))
    num = [i for i in range(16709)]
    vallist = set(num) - set(trainlist)
    vallist = list(vallist)
    for i in trainlist:
        x_training1.append(x_train1[i])
        x_training2.append(x_train2[i])
        x_training3.append(x_train3[i])
        y_training.append(y_train[i])
        groups_training.append(groups_train[i])
    for i in vallist:
        x_val1.append(x_train1[i])
        x_val2.append(x_train2[i])
        x_val3.append(x_train3[i])
        y_val.append(y_train[i])
        groups_val.append(groups_train[i])
    x_training1 = np.array(x_training1, dtype="float32").transpose((0, 2, 1))
    x_training2 = np.array(x_training2, dtype="float32").transpose((0, 2, 1))
    x_training3 = np.array(x_training3, dtype="float32").transpose((0, 2, 1))
    y_training = np.array(y_training, dtype="float32")
    x_val1 = np.array(x_val1, dtype="float32").transpose((0, 2, 1))
    x_val2 = np.array(x_val2, dtype="float32").transpose((0, 2, 1))
    x_val3 = np.array(x_val3, dtype="float32").transpose((0, 2, 1))
    y_val = np.array(y_val, dtype="float32")
    x_test1,x_test2,x_test3 = [],[],[]
    o_test, y_test = apnea_ecg["o_test"], apnea_ecg["y_test"]
    groups_test = apnea_ecg["groups_test"]
    for i in range(len(o_test)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_test[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_test1.append([rri_interp_signal, ampl_interp_signal])
        x_test2.append([rri_interp_signal[180:720], ampl_interp_signal[180:720]])
        x_test3.append([rri_interp_signal[360:540], ampl_interp_signal[360:540]])
    x_test1 = np.array(x_test1, dtype="float32").transpose((0, 2, 1))
    x_test2 = np.array(x_test2, dtype="float32").transpose((0, 2, 1))
    x_test3 = np.array(x_test3, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")

    return x_training1, x_training2, x_training3, y_training, groups_training, x_val1, x_val2, \
           x_val3, y_val, groups_val, x_test1, x_test2, x_test3, y_test, groups_test


def lr_schedule(epoch, lr):
    if epoch > 70 and (epoch - 1) % 10 == 0:
        lr *= 0.1
    print("Learning rate: ", lr)
    return lr


# Setting up the architecture
def create_model(input_a_shape, input_b_shape, input_c_shape, weight=1e-3):
    # SA-CNN-3
    input1 = Input(shape=input_a_shape)
    x1 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input1)
    x1 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=3, padding="same")(x1)
    x1 = Conv1D(32, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=5, padding="same")(x1)

    # SA-CNN-2
    input2 = Input(shape=input_b_shape)
    x2 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input2)
    x2 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x2)
    x2 = MaxPooling1D(pool_size=3, padding="same")(x2)
    x2 = Conv1D(32, kernel_size=11, strides=3, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x2)

    # SA-CNN-1
    input3 = Input(shape=input_c_shape)
    x3 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input3)
    x3 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x3)
    x3 = MaxPooling1D(pool_size=3, padding="same")(x3)
    x3 = Conv1D(32, kernel_size=1, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x3)

    # Channel-wise attention module
    concat = keras.layers.concatenate([x1, x2, x3], name="Concat_Layer", axis=-1)
    squeeze = GlobalAveragePooling1D()(concat)
    excitation = Dense(48, activation='relu')(squeeze)
    excitation = Dense(96, activation='sigmoid')(excitation)
    excitation = Reshape((1, 96))(excitation)
    scale = multiply([concat, excitation])
    x = GlobalAveragePooling1D()(scale)
    dp = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax', name="Output_Layer")(dp)
    model = Model(inputs=[input1, input2, input3], outputs=outputs)
    return model

### Load data

In [ ]:
# load_data
path = "apnea-ecg.pkl"
x_train1, x_train2, x_train3, y_train, groups_train, x_val1, x_val2,\
x_val3, y_val, groups_val, x_test1, x_test2, x_test3, y_test, groups_test = load_data(path)
y_train = keras.utils.to_categorical(y_train, num_classes=2)  # Convert to two categories
y_val = keras.utils.to_categorical(y_val, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)
print('input_shape', x_train1.shape, x_train2.shape, x_train3.shape)

### Training the model

In [ ]:
# training
model = create_model(x_train1.shape[1:], x_train2.shape[1:], x_train3.shape[1:])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath = 'weights.best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
lr_scheduler = LearningRateScheduler(lr_schedule)
callbacks_list = [lr_scheduler, checkpoint]
history = model.fit([x_train1, x_train2, x_train3], y_train, batch_size=128, epochs=100,
                    validation_data=([x_val1, x_val2, x_val3], y_val), callbacks=callbacks_list)

input_shape (11696, 900, 2) (11696, 540, 2) (11696, 180, 2)
Learning rate:  0.0010000000474974513
Epoch 1/100
92/92 [==============================] - ETA: 0s - loss: 0.9278 - accuracy: 0.6585
Epoch 1: val_accuracy improved from -inf to 0.79812, saving model to weights.best.hdf5
92/92 [==============================] - 43s 416ms/step - loss: 0.9278 - accuracy: 0.6585 - val_loss: 0.7002 - val_accuracy: 0.7981 - lr: 0.0010
Learning rate:  0.0010000000474974513
Epoch 2/100
92/92 [==============================] - ETA: 0s - loss: 0.6351 - accuracy: 0.8210
Epoch 2: val_accuracy improved from 0.79812 to 0.85059, saving model to weights.best.hdf5
92/92 [==============================] - 37s 409ms/step - loss: 0.6351 - accuracy: 0.8210 - val_loss: 0.5454 - val_accuracy: 0.8506 - lr: 0.0010
Learning rate:  0.0010000000474974513
Epoch 3/100
92/92 [==============================] - ETA: 0s - loss: 0.5572 - accuracy: 0.8393
Epoch 3: val_accuracy did not improve from 0.85059
92/92 [================

### Verifying perfomance with testing dataset

In [ ]:
# test
filepath = './weights.best.hdf5'
model = load_model(filepath)
loss, accuracy = model.evaluate([x_test1, x_test2, x_test3], y_test)
# save prediction score
y_score = model.predict([x_test1, x_test2, x_test3])
output = pd.DataFrame({"y_true": y_test[:, 1], "y_score": y_score[:, 1], "subject": groups_test})
output.to_csv("SE-MSCNN.csv", index=False)
y_true, y_pred = np.argmax(y_test, axis=-1), np.argmax(model.predict([x_test1, x_test2, x_test3], batch_size=1024, verbose=1), axis=-1)
C = confusion_matrix(y_true, y_pred, labels=(1, 0))
TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
acc, sn, sp = 1. * (TP + TN) / (TP + TN + FP + FN), 1. * TP / (TP + FN), 1. * TN / (TN + FP)
f1 = f1_score(y_true, y_pred, average='binary')
print("acc: {}, sn: {}, sp: {}, f1: {}".format(acc, sn, sp, f1))

### Comparing with other baseline models

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score

base_dir = "output"

# Table 2
output = []
svm = '/content/SVM.csv'
lr = '/content/LR.csv'
knn = '/content/KNN.csv'
mlp = '/content/MLP.csv'
se_mscnn = '/content/SE-MSCNN.csv'
methods = [svm, lr, knn, mlp, se_mscnn]
for method in methods:
    df = pd.read_csv(method, header=0)
    df["y_pred"] = df["y_score"] > 0.5
    df = df.groupby(by="subject").apply(lambda d: d["y_pred"].mean() * 60)
    df.name = method
    output.append(df)
output = pd.concat(output, axis=1)

with open("additional-information.txt", "r") as f:
    original = []
    for line in f:
        rows = line.strip().split("\t")
        if len(rows) == 12:
            if rows[0].startswith("x"):
                original.append([rows[0], float(rows[3]) / float(rows[1]) * 60])
original = pd.DataFrame(original, columns=["subject", "original"])
original = original.set_index("subject")
all = pd.concat((output, original), axis=1)
corr = all.corr()
all1 = all.applymap(lambda a: int(a > 5))
result = []

# Iteratively calculate confusion matrix
for method in methods:
    C = confusion_matrix(all1["original"], all1[method], labels=(1, 0))
    TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
    acc, sn, sp = 1. * (TP + TN) / (TP + TN + FP + FN), 1. * TP / (TP + FN), 1. * TN / (TN + FP)
    auc = roc_auc_score(all["original"] > 5, all[method])
    result.append([method, acc * 100, sn * 100, sp * 100, auc, corr["original"][method]])

    # a pictorial representation (heatmap) of the confusion matrix
    plt.figure()
    plt.imshow(C, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {method}')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, [1, 0])
    plt.yticks(tick_marks, [1, 0])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    for i in range(2):
        for j in range(2):
            plt.text(j, i, f'{C[i, j]}', horizontalalignment='center', color='white' if C[i, j] > C.max() / 2 else 'black')

    plt.show()

